In [10]:
import glob
import segyio
import segysak
import pandas as pd
import numpy as np
import xarray as xr
from devito import configuration, VectorTimeFunction, TensorTimeFunction
from examples.seismic import AcquisitionGeometry
from examples.seismic.elastic import ElasticWaveSolver
from scratch.util import (
                    CreateSeismicModel,
                    nn_interp_coords,
                    plot_rec_src,
                    plot_seis_data)

In [2]:
configuration['ignore-unknowns'] = True

In [3]:
#Закоменти, чтобы выполнять на CPU
from devito import configuration

#(a) using one GPU
#(b) using multi-GPUs with MPI (requires turning the notebook into a python script)
configuration['platform'] = 'nvidiaX'
configuration['compiler'] = 'pgcc'
configuration['language'] = 'openacc'

In [4]:
sc_path = '2D_Scenarios'
scenarios = glob.glob(sc_path+'/sc*')

df_ins = pd.read_csv(sc_path+'/instruments.txt', sep='\t')
df_ins['Z'] += 2
df_ins['X'] = np.linspace(0,7950, 319)

constraints = {"Vp": 1800, "Vs": 750, "Rho" : 1500}

In [5]:
for i, scenario in enumerate(scenarios[1:]): #single check
# for i, scenario in enumerate(scenarios): #whole
    readsgy = lambda x : xr.open_dataset(x,
                                         dim_byte_fields={"cdp" : 1},
                                         extra_byte_fields={'cdp_x':181, 'cdp_y':185}
                                        )
    el_pars = {file.split('/')[-1].split(' ')[0] : readsgy(file) for file in glob.glob(scenario+'/*.sgy')}
    # plot_hist_pars(el_pars, ignore_zero=True) # гистограммы параметров перед корректировкой
    for k, v in el_pars.items():
        el_pars[k] = el_pars[k].where(((el_pars[k] > constraints[k]) | (el_pars[k].samples>100) | (el_pars[k] == 0)), constraints[k])
    # plot_hist_pars(el_pars, ignore_zero=True)  # гистограммы параметров после корректировки

    # привычный формат
    rho_data = (el_pars["Rho"].data/1000).to_numpy()
    vp_data = (el_pars["Vp"].data/1000).to_numpy()
    vs_data = (el_pars["Vs"].data/1000).to_numpy()

    # Верхний слой со свойствами воды
    rho_data[rho_data==0] = 1.
    vp_data[vp_data==0] = 1.5
    vs_data[vs_data==0] = 0.0

    # сетка
    dim_vectors = ((el_pars['Rho'].cdp.data-1)*25, (el_pars['Rho'].samples.data))
    spacing = (1, 1) # z из header_rho.loc["TRACE_SAMPLE_INTERVAL", "mean"]
    origin = (0, 0)
    nbl = 40
    so = 8
    
    # инт данные
    rho_data_int = nn_interp_coords(rho_data, origin, (7950, 1000), spacing, dim_vectors)
    vp_data_int = nn_interp_coords(vp_data, origin, (7950, 1000), spacing, dim_vectors)
    vs_data_int = nn_interp_coords(vs_data, origin, (7950, 1000), spacing, dim_vectors)

    # модель
    model = CreateSeismicModel(origin=origin,
                           spacing=spacing,
                           shape=vp_data_int.shape,
                           vp=vp_data_int,
                           vs=vs_data_int,
                           rho=rho_data_int,
                           so=so,
                           nbl=nbl,
                           bcs='mask',
                          )
    
    # геометрия
    t0=0.
    tn=2000.
    f0=0.015

    nsrc = 1
    src_coordinates = np.empty((nsrc, 2))
    src_coordinates[:] = ([df_ins['X'][159], 0])

    nrec = df_ins.shape[0]
    rec_coordinates = np.empty((nrec, 2))
    rec_coordinates[:,0] = df_ins['X']
    rec_coordinates[:,1] = df_ins['Z'] 

    geometry = AcquisitionGeometry(model, rec_coordinates, src_coordinates, t0, tn, f0=f0, src_type='Ricker')
    
    # солвер
    v = VectorTimeFunction(name='v', grid=model.grid, space_order=so, time_order=2)
    tau = TensorTimeFunction(name='t', grid=model.grid, space_order=so, time_order=2)
    solver = ElasticWaveSolver(model, geometry, space_order=so, v=v, tau=tau)
    
    # оператор
    rec_p, rec_v, v, tau, summary = solver.forward()
    
    # выгрузка в sgy
    dt_r = 1
    inheader = segysak.segy.segy_header_scrape(scenarios[1]+'/Vs 2D 1.sgy')
    rec_v = rec_v.resample(dt=dt_r)
    
    segyio.tools.from_array2D('Results/2d_vankor_'+scenarios[1].split('/')[1]+'.sgy', rec_v.data.T, dt=dt_r*10**3)
    with segyio.open('Results/2d_vankor'+scenarios[1].split('/')[1]+'.sgy', 'r+') as f:
        for j in range(len(f.header)):
            f.header[j] = {segyio.TraceField.CDP: j,
                           segyio.TraceField.CDP_X: np.array(inheader['CDP_X'][j]),
                           segyio.TraceField.CDP_Y: np.array(inheader['CDP_Y'][j]),
                           segyio.TraceField.ReceiverGroupElevation: np.array(df_ins['Z'][j], dtype = int),
                           segyio.TraceField.ElevationScalar : np.array(inheader['ElevationScalar'][j]),
                           segyio.TraceField.SourceGroupScalar : 1,
                           segyio.TraceField.SourceX : np.array(df_ins['X'][159], dtype = int)
                          }
    #Доделать, чтобы шот отображался еще и в названии

  0%|          | 0.00/1.00 [00:00<?, ? trace-chunks/s]

  0%|          | 0.00/319 [00:00<?, ? traces/s]

  0%|          | 0.00/1.00 [00:00<?, ? trace-chunks/s]

  0%|          | 0.00/319 [00:00<?, ? traces/s]

  0%|          | 0.00/1.00 [00:00<?, ? trace-chunks/s]

  0%|          | 0.00/319 [00:00<?, ? traces/s]

Loading Traces:   0%|          | 0.00/319 [00:00<?, ? traces/s]

Loading Traces:   0%|          | 0.00/319 [00:00<?, ? traces/s]

Loading Traces:   0%|          | 0.00/319 [00:00<?, ? traces/s]

Operator `initdamp` ran in 0.07 s
Operator `ForwardElastic` ran in 28.97 s


In [ ]:
plot_rec_src(model=model, data_type='lam', src_coords=src_coordinates, rec_coords=rec_coordinates, xrange=(0,2), yrange=(0.1,0))

In [ ]:
plot_seis_data(rec_coordinates=rec_coordinates,rec_data=rec1.data, t0=t0, tn=tn)